In [2]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import pandas as pd
import os
import math

class MyModel_0(nn.Module):
    def __init__(self):#在这里定义网络层
        super(MyModel_0, self).__init__()
        self.liner1=nn.Linear(755,1,bias=False)
        nn.init.zeros_(self.liner1.weight)

    def forward(self,x):#前向传播
        output=self.liner1(x)
        return output
    
class MyModel_1(nn.Module):
    def __init__(self):#在这里定义网络层
        super(MyModel_1, self).__init__()
        self.liner1=nn.Linear(755,1,bias=False)
        nn.init.zeros_(self.liner1.weight)

    def forward(self,x):#前向传播
        output=self.liner1(x)
        return output

class CustomLoss(torch.nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, output, target):
        loss = torch.mean(0.5*(output - target)**2)   # 定义自己的损失函数，这里是MSE
        return loss
model_0=MyModel_0()#建立模型
model_1=MyModel_1()#建立模型
loss_fn=CustomLoss()#构建损失函数
optimizer_0=optim.Adam(model_0.parameters(),lr=0.001)#构建优化器(模型参数,学习率)
optimizer_1=optim.Adam(model_1.parameters(),lr=0.001)#构建优化器(模型参数,学习率)
B=5000
Epoch=5000
danwei_b=B/(1241*30)
experience=[]
nt=[0,0,0,0]
loss_abc=[]
rootpath='C:\\Users\\98306\\Desktop\\实验\\训练数据\\500'
for i in range(Epoch):
    print('epoch:',i)
    idx=random.randrange(0,100)
    roadpath=str(idx)+'_road.xlsx'
    roadpath=os.path.join(rootpath,roadpath)
    road=pd.read_excel(roadpath)
    road=road.to_numpy()
    
    bpath=str(idx)+'_b.xlsx'
    bpath=os.path.join(rootpath,bpath)
    b=pd.read_excel(bpath,header=None)
    b=b.to_numpy().flatten()
    
    path=r'C:\Users\98306\Desktop\实验\Real map\poi次数分布(总次数为1241).xlsx'
    df=pd.read_excel(path)
    arr_df = df.to_numpy().flatten()

    B_i=B
    res=0#记录每轮最大的TD-error
    total_value=0#每轮获得的总价值
    
    for j in range(road.shape[0]):
        for k in range(road.shape[1]):
            if arr_df[k]==0:
                road[j,k]=0
        if road[j].sum()*danwei_b<b[j]:
            continue
        pi=0
        reward=0
        s= np.array([]).astype(np.double)
        s= np.append(s,1)
        s= np.append(s,j)
        s= np.append(s, road[j].sum())
        s= np.append(s,b[j])
        s= np.concatenate((s,arr_df))
        s= np.append(s,B_i)
        epsilon=0.1
        lamda=random.uniform(0, 1)
        if lamda<epsilon/2:#随机不选
            pi=0
            reward=0
            nt[0]+=1
        elif lamda<epsilon:#随机选择
            pi=1
            reward=road[j].sum()
            nt[1]+=1
        else:
            with torch.no_grad():
                t0=model_0(torch.Tensor(s))
                t1=model_1(torch.Tensor(s))
                t0=t0.detach().numpy()
                t1=t1.detach().numpy()
                
                if t0[0]<t1[0]:
                    pi=1
                    reward=road[j].sum()
                    nt[2]+=1
                else:
                    pi=0
                    reward=0
                    nt[3]+=1

        if j>=1:#存入经验回放数组
            exper=[]
            exper.append(s_old)
            exper.append(pi_old)
            exper.append(reward_old)
            exper.append(s)
            exper.append(pi)
            experience.append(exper)
#         if j==road.shape[0]-1:
#             exper=[]
#             exper.append(s)
#             exper.append(pi)
#             exper.append(reward)
#             exper.append(np.array([0]))
#             exper.append(0)
#             experience.append(exper)
            
        s_old=s
        pi_old=pi
        reward_old=reward
        
        if pi==1:
            for k in range(road.shape[1]):
                if road[j,k]>0:
                    arr_df[k]-=1
            B_i-=road[j].sum()*danwei_b
            

        
        total_value+=reward
        if len(experience)>=1000:
            experience.pop(0)
        if len(experience)>100:
            idx=random.randrange(0,len(experience))#随机抽取一条经验
            replay_s_old=experience[idx][0]#S
            replay_pi_old=experience[idx][1]#A
            replay_reward=experience[idx][2]#R
            replay_s=experience[idx][3]#S
            replay_pi=experience[idx][4]#A
#             if replay_s.shape[0]==1:
#                 x_train=torch.Tensor(replay_s_old)
#                 real=torch.Tensor(np.array([replay_reward]))
#                 if replay_pi_old==0:
#                     pred=model_0(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_0.zero_grad()
#                     loss.backward()
#                     optimizer_0.step()
#                     res=max(res,loss.item())
#                 else:
#                     pred=model_1(x_train)
#                     loss=loss_fn(pred,real)
                    
#                     optimizer_1.zero_grad()
#                     loss.backward()
#                     optimizer_1.step()
#                     res=max(res,loss.item())
#             else:
            if replay_s.shape[0]!=1:
                x_train=torch.Tensor(replay_s_old)
                y_train=torch.Tensor(replay_s)
                if replay_pi_old==0 and replay_pi==0:
#                     print("111")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==0 and replay_pi==1:
#                     print("222")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_0(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_0.zero_grad()
                    loss.backward()
                    optimizer_0.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==0:
#                     print("333")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_0(y_train)
#                         print(real,torch.Tensor(np.array([replay_reward])),model_0(y_train))
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
                elif replay_pi_old==1 and replay_pi==1:
#                     print("444")
                    with torch.no_grad():
                        real=torch.Tensor(np.array([replay_reward]))+model_1(y_train)
                    pred=model_1(x_train)
                    loss=loss_fn(pred,real)
        
                    
                    optimizer_1.zero_grad()
                    loss.backward()
                    optimizer_1.step()
                    res=max(res,loss.item())
            with torch.no_grad():
                if replay_pi_old==1:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,755):
                            if k==1 or k==3:
                                if param1[0][k] > param0[0][k]:
                                    param1[0][k]=param0[0][k].clone()
                elif replay_pi_old==0:
                    for param0, param1 in zip(model_0.parameters(), model_1.parameters()):
                        for k in range(1,755):
                            if k==1 or 3:
                                if param1[0][k] > param0[0][k]:
                                    param0[0][k]=param1[0][k].clone()      
                
    print("Total_value:",total_value)
    print("剩余预算:",B_i)
    print("TD-error",res)
    print(" ")
    loss_abc.append(math.sqrt(res))
print(nt)


epoch: 0


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\98306\\Desktop\\实验\\训练数据\\500\\98_road.xlsx'